<a href="https://colab.research.google.com/github/SaiTejaMutchi/airline_case_study/blob/main/airline_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part A

In [32]:
import pandas as pd

In [33]:
from google.colab import files
uploaded = files.upload()

In [34]:
# Given excel was split into csv files as per csv files
# importing data
ac_characteristics_df = pd.read_csv('ac_characteristics.csv')
city_pairs_df = pd.read_csv('city_pairs.csv')
operations_df=pd.read_csv('operations.csv')

In [35]:
ac_characteristics_df

,Aircraft Type,Range (km),Average Speed (km/h),Number of Seats,Costs per flight hour
0,A320,5000,800,150,5000
1,A330,8000,900,250,7500
2,B737,5000,800,150,5100
3,B747,10000,900,350,12000
4,Q400,1500,750,90,3500
5,ATR72,1000,650,75,2750


In [36]:
# ac_charactersrics_df
ac_type_dict={}
for i in range(len(ac_characteristics_df)):
  my_list=list(ac_characteristics_df.iloc[i])
  ac_type_dict[my_list[0]]=my_list[1:]
ac_type_dict
# this dictionary has ac_type as key and value as list of Range, Avg. speed, Number of seats, Costs per flight hour

{'A320': [5000, 800, 150, 5000],
 'A330': [8000, 900, 250, 7500],
 'ATR72': [1000, 650, 75, 2750],
 'B737': [5000, 800, 150, 5100],
 'B747': [10000, 900, 350, 12000],
 'Q400': [1500, 750, 90, 3500]}

In [37]:
operations_df

,Aircraft Name,Aircraft Type,Jan-14,Feb-14,Mar-14,Apr-14,May-14,Jun-14,Jul-14,Aug-14,Sep-14,Oct-14,Nov-14,Dec-14
0,A,A320,250,251,248,253,243,245,248,246,256,256,251,252
1,B,A330,300,301,298,303,293,295,298,296,306,306,301,302
2,C,B737,245,246,243,248,238,240,243,241,251,251,246,247
3,D,B747,310,311,308,313,303,305,308,306,316,316,311,312
4,E,Q400,200,201,198,203,193,195,198,196,206,206,201,202
5,F,ATR72,195,196,193,198,188,190,193,191,201,201,196,197
6,G,B747,300,301,298,303,293,295,298,296,306,306,301,302
7,H,Q400,195,196,193,198,188,190,193,191,201,201,196,197
8,I,ATR72,200,201,198,203,193,195,198,196,206,206,201,202
9,J,A330,310,311,308,313,303,305,308,306,316,316,311,312


In [38]:
# operations_df, getting total hours of a given ac type
total_hours_dict={}
for i in range(len(operations_df)):
  my_list=list(operations_df.iloc[i])
  if my_list[1] in total_hours_dict:
    total_hours_dict[my_list[1]]+= sum(my_list[2:])
  else: 
    total_hours_dict[my_list[1]]=sum(my_list[2:])
total_hours_dict

{'A320': 14875,
 'A330': 21834,
 'ATR72': 9536,
 'B737': 11816,
 'B747': 11037,
 'Q400': 9476}

## Estimating total cost by aircraft type for the year

In [39]:
# Estimating total cost:
totalcost_dict={}
for i in total_hours_dict:
  totalcost_dict[i]=ac_type_dict[i][-1]*total_hours_dict[i]
totalcost_dict

{'A320': 74375000,
 'A330': 163755000,
 'ATR72': 26224000,
 'B737': 60261600,
 'B747': 132444000,
 'Q400': 33166000}

## Aircraft type which has the lowest cost per seat per km flown

In [40]:
# lowest_Cost
lowest_Cost={}
for i in totalcost_dict:
  lowest_Cost[i]=totalcost_dict[i]/(ac_type_dict[i][-2]*ac_type_dict[i][1]*total_hours_dict[i])
lowest_Cost_ac=min(lowest_Cost, key=lowest_Cost.get)
print(str(lowest_Cost_ac)+': {}'.format(lowest_Cost[lowest_Cost_ac]))

A330: 0.03333333333333333


# Part B

In [41]:
# we have city pairs their data
city_pairs_dict={}
for i in range(len(city_pairs_df)):
  pair=city_pairs_df.iloc[i][0]+city_pairs_df.iloc[i][1]
  city_pairs_dict[pair]=[city_pairs_df.iloc[i][2],city_pairs_df.iloc[i][3]]
city_pairs_dict


{'AA BB': [420, 3000],
 'AA DD': [300, 1000],
 'BB CC': [450, 6500],
 'CC AA': [300, 400]}

- city_pairs_dict represents the city pairs with Pass. Demand and Distance.
- We need to get the aircrafts which have range higher than or equal to the distance between the city pair.
- from ac_type_dict , we use no. of seats in order to estimate no. of trips required for a given city pair and air craft type

- For every city pair we take ac_type get the lowest cost one by using no.of trips and total cost

In [42]:
import sys
import math

fleet=[]
city_pairs_ac_dict={}
for city_pair in city_pairs_dict:

  # getting aircrafts whose range is greater than the distance of given city pair
  range_ac=[ac_type for ac_type in ac_type_dict if ac_type_dict[ac_type][0]>=city_pairs_dict[city_pair][1]]
  lowest_cost_for_city_pair = sys.maxsize
  lowest_cost_type_city_pair = None
  
  # iterating through eligible ac_types in order to get most suited one
  for ac_type in range_ac:

# dividing city pairs demand by seating capacity of ac type inorder to find no. of trips required for a given ac type
    no_trips=math.ceil(1.0*city_pairs_dict[city_pair][0]/ac_type_dict[ac_type][2])

# getting cost per trip, we take the distance between the cities and divide it by the average speed to get the time taken for the trip (in hours)
# then multiply by cost per hour
    cost_for_trip = (1.0 * city_pairs_dict[city_pair][1]/ac_type_dict[ac_type][1])*ac_type_dict[ac_type][3]
    totalcost=cost_for_trip*no_trips

    # finding lowest cost ac for a given city pair
    if totalcost < lowest_cost_for_city_pair:
      lowest_cost_for_city_pair = totalcost
      lowest_cost_type_city_pair = ac_type

  # appending the lowest cost ac for given pair to the fleet 
  if lowest_cost_type_city_pair not in fleet:
    fleet.append(lowest_cost_type_city_pair)

  # noting total cost & suited ac_type for a given city pair
  city_pairs_ac_dict[city_pair]=[lowest_cost_type_city_pair, lowest_cost_for_city_pair]



In [43]:
# part B solution is in the city_pairs_ac_dict with type as first value and total as second value
city_pairs_ac_dict

{'AA BB': ['A330', 50000.0],
 'AA DD': ['A320', 12500.0],
 'BB CC': ['A330', 108333.33333333333],
 'CC AA': ['A320', 5000.0]}

In [44]:
# The best suited aircrafts are:
fleet

['A330', 'A320']